In [1]:
import argparse
import math
import time
import json
import dill as pickle
from tqdm import tqdm

import torch
import torch.nn.functional as F
import torch.optim as optim
from torchtext.data import Field, Dataset, BucketIterator
from torchtext.datasets import TranslationDataset

import transformer.Constants as Constants
from transformer.Models import Transformer,Encoder,Encoder_Sub,get_pad_mask
from transformer.Optim import ScheduledOptim


import torch.nn as nn
import numpy as np


In [2]:

# device = torch.device('cuda' if torch.cuda.is_available()  else 'cpu')
# # N是批量大小，D_in 是输入维度，H 是隐藏维度， D_out 是输出维度
# # data=torch.randn()


# # input_data = data.to
# n_src_vocab=1000
# src_pad_idx=0

# d_word_vec=512
# d_model=512
# d_inner=2048
# n_layers=6
# n_head=8
# d_k=64
# d_v=64
# dropout=0.1

# n_position=200 #位置嵌入向量应该等于maxlen

# sup_enc = Encoder(n_src_vocab = n_src_vocab, n_position = n_position,
#         d_word_vec = d_word_vec, d_model=d_model, d_inner=d_inner,
#         n_layers=n_layers, n_head=n_head, d_k=d_k, d_v=d_v,
#         pad_idx=src_pad_idx, dropout=dropout)
# # sub_enc

# # input=torch.autograd.Variable(input)
# # sup_enc(input)

# #batch,seq_len
# src_seq=torch.arange(0,15).view(3,5).long()    #三个句子，每个句子有两个词
# # src_seq=torch.cat([src_seq,torch.zeros(3,5)],1)
# mask=get_pad_mask(src_seq, src_pad_idx)

# src_out=sup_enc(src_seq,mask)
# src_out[0].shape

In [1]:
import torch
import numpy as np

input_datas = torch.arange(0,2*3*4,dtype=torch.float32).view(2,3,4)
print(input_datas)
a=input_datas.max(dim=1).values
b=input_datas.mean(dim=1)
torch.cat([a,b],1).shape

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]],

        [[12., 13., 14., 15.],
         [16., 17., 18., 19.],
         [20., 21., 22., 23.]]])


torch.Size([2, 8])

In [4]:
class Dual_Encoder(nn.Module):
    def __init__(
            self, n_src_vocab,src_pad_idx,class_num,
            d_word_vec=512, d_model=512, d_inner=2048,
            n_layers=6, n_head=8, d_k=64, d_v=64, dropout=0.1, n_position=200,
            ):

        super().__init__()

        self.pad_idx = src_pad_idx
        #
        self.sup_enc = Encoder(n_src_vocab = n_src_vocab, n_position = n_position,
                d_word_vec = d_word_vec, d_model=d_model, d_inner=d_inner,
                n_layers=n_layers, n_head=n_head, d_k=d_k, d_v=d_v,
                pad_idx=src_pad_idx, dropout=dropout)

        self.sub_enc=Encoder_Sub(
                    d_model=d_model, d_inner=d_inner,
                    n_layers=n_layers, n_head=n_head, d_k=d_k, d_v=d_v,
                    dropout=dropout)
        
        #batch,len,emb_sice
        self.fc = nn.Linear(d_word_vec*2, class_num)

    #txtEmb = 
    # input_ids（batch,len）: txt a ,b进行cls-pad-seq填充后的句子,id表示
    # index_mask（bacth,len）：标识A-B句子对应位置的mask,1-A,2-B，
    def forward(self,txta_ids,txta_index,txtb_ids,txtb_index):
        # txta_ids,txta_index:batch,len
        inputa_mask = get_pad_mask(txta_ids, 0)
        inputb_mask = get_pad_mask(txtb_ids, 0)

        # new_ipt_mask = torch.squeeze(input_mask,dim=1)
        #
        embA_out = self.sup_enc(txta_ids,inputa_mask)[0]
        embB_out = self.sup_enc(txtb_ids,inputb_mask)[0]
        #batch,len,emb_size
        inter_emb_out = self.sub_enc(embB_out,inputb_mask,embA_out)[0]
        #batch,len,emb_size
        mean_emb = inter_emb_out.max(dim=1).values
        max_emb = inter_emb_out.mean(dim=1)
        #batch,emb_size
        cat_out = torch.cat([mean_emb,max_emb],1)
        #batch,emb_size*2
        fc_out = self.fc(cat_out)
        #batch,cls_num
        logit = F.log_softmax(fc_out, dim=1)
        return logit#batch,cls_num

In [45]:
from transformers import BertTokenizer, BertModel
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

TypeError: __init__() got an unexpected keyword argument 'return_dict'

In [5]:
import jsonlines
import copy
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def load_data_from_file(path):
    with open(path,mode='r+',encoding='utf8') as f:
        data_list=[]
        for step,item in enumerate(jsonlines.Reader(f)):
            if item['gold_label']=='-':
                continue
            data_list.append({'index':step,'sentence1':item['sentence1'],'sentence2':item['sentence2'],'gold_label':item['gold_label']})
    return data_list


# 创建数据读取器data_reader
def snli_data_reader(data_path,label_dict,padding_idx=0):
#     with open(data_path, "rb") as pkl:
    datas=load_data_from_file(data_path)
    # newdatas=datas.copy()
    # 打乱数据
    # datas=load_dataset(data_path)
    # random.shuffle(newdatas)
    # 开始获取每个数据和标签
    #分别处理'premises', 'hypotheses'
    #1.先分别计算出最大长度
    premises_lengths=[]
    hypotheses_lengths=[]
    #
    all_data_list=[]
    label_list=[]
    all_data_index=[]
#     all_data_mask=[]
    #
    for item in datas:
        premises_lengths.append(len(item['sentence1']))
        hypotheses_lengths.append(len(item['sentence2']))
        label_list.append(label_dict[item['gold_label']])
        
        token_itemA = tokenizer(item['sentence1'],return_tensors="pt")
        token_itemB = tokenizer(item['sentence2'],return_tensors="pt")
        
        all_data_list.append((token_itemA.input_ids,token_itemB.input_ids))#A,B输入序列的id
        
        #ab句子对应的mask位置0代表a巨，1代表b句的位置
        all_data_index.append((token_itemA.token_type_ids,token_itemB.token_type_ids))
        
#         all_data_mask.append(token_item.attention_mask)
    
    #最大长度得算上cls+seq
    #分别处理a，b,找到两者最大的长度
    maxlen=max(max(premises_lengths),max(hypotheses_lengths))+2
    
    pad_data_list=[]
    pad_index_list=[]
#     pad_mask_list=[]
    
    #2.对数据进行pad填充
    for data_id,data_item in enumerate(zip(all_data_list,all_data_index)):
        p_data,data_index=data_item
#         pad_data,pad_mask = len_process(p_data,data_mask,maxlen,padding_idx)#填充pad值
        #填充
        a_pad_value = torch.full([1, maxlen-p_data[0].shape[1]], padding_idx,dtype = torch.int64)
        b_pad_value = torch.full([1, maxlen-p_data[1].shape[1]], padding_idx,dtype = torch.int64)
        
#         pad_mask = torch.cat([data_mask,pad_value],1)
        
        a_pad_index = torch.cat([data_index[0],a_pad_value],1).view(-1)
        b_pad_index = torch.cat([data_index[1],b_pad_value],1).view(-1)
        
        a_pad_data = torch.cat([p_data[0],a_pad_value],1).view(-1)
        b_pad_data = torch.cat([p_data[1],b_pad_value],1).view(-1)
        
        pad_data_list.append((a_pad_data,b_pad_data))
        pad_index_list.append((a_pad_index,b_pad_index))
#         pad_mask_list.append(pad_mask)
    return pad_data_list,pad_index_list,label_list,maxlen


class SNLI_Dataset(Dataset):
    def __init__(self,filepath,label_dict,pad_id):
#         pad_data_list,pad_mask_list,all_data_index,label_list
        self.data,self.data_index,self.cls,self.maxlen=snli_data_reader(filepath,label_dict,pad_id)
        self.len = len(self.cls)
        
    def data_max_len(self):
        return self.maxlen
    
    def __getitem__(self, index):
        return (self.data[index][0],self.data_index[index][0],#premise
                self.data[index][1],self.data_index[index][1],#hypothese
                self.cls[index])
    
    def __len__(self):
        return self.len


In [20]:
# train_path='../../data/raw_data/snli_1.0_train.jsonl'
# test_path='../../data/raw_data/snli_1.0_test.jsonl'
# dev_path='../../data/raw_data/snli_1.0_dev.jsonl'
# # pad_data_list,pad_mask_list,all_data_index,label_list=snli_data_reader(dev_path,0)
# label_dict = {  "entailment": 0,
#                 "neutral": 1,
#                 "contradiction": 2}

# snli_dataset=SNLI_Dataset(test_path,label_dict,0)
# max_len=snli_dataset.data_max_len()#最大长度


In [21]:
# batch_size=1
# dataLoader=DataLoader(snli_dataset, batch_size=batch_size, shuffle=True)

In [28]:
# input_data = data.to

print(max_len)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

net = Dual_Encoder(n_src_vocab,src_pad_idx,class_num,
            d_word_vec=d_word_vec, d_model=d_model, d_inner=d_inner,
            n_layers=n_layers, n_head=n_head, d_k=d_k, d_v=d_v, dropout=dropout, n_position=n_position,
            ).to(device)

# for i, (txta_ids,txta_index,txtb_ids,txtb_index,cls) in enumerate(dataLoader):
   
#     for i in (txta_ids,txta_index,txtb_ids,txtb_index,cls):
#         print(i.shape)
        
#     out = model(txta_ids,txta_index,txtb_ids,txtb_index)
#     print(out.shape)
    
#     inputa_mask = get_pad_mask(txta_ids, 0)
#     inputb_mask = get_pad_mask(txta_index, 0)
    
# #     print(input_mask)
# #     new_ipt_mask = torch.squeeze(input_mask,dim=1)
    
#     embA_out = sup_enc(txta_ids,inputa_mask)[0]
#     embB_out = sup_enc(txtb_ids,inputb_mask)[0]
#     print(embA_out.shape,embB_out.shape)
#     inter_emb_out = sub_enc(embB_out,inputb_mask,embA_out)
    
#     print(inter_emb_out[0].shape)



#     break

267
cpu


In [6]:
import logging
def log(name='info',log_path="info.log",level=logging.DEBUG):
    #创建logger，如果参数为空则返回root logger
    logger = logging.getLogger(name)
#     level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
    logger.setLevel(level)  #设置logger日志等级

    #这里进行判断，如果logger.handlers列表为空，则添加，否则，直接去写日志
    if not logger.handlers:
        #创建handler
        fh = logging.FileHandler(log_path,encoding="utf-8")
        ch = logging.StreamHandler()

        #设置输出日志格式
        formatter = logging.Formatter(
            fmt="%(lineno)d %(asctime)s %(name)s %(filename)s %(levelname)s %(message)s ",
            datefmt="%Y/%m/%d %X"
            )

        #为handler指定输出格式
        fh.setFormatter(formatter)
        ch.setFormatter(formatter)

        #为logger添加的日志处理器
        logger.addHandler(fh)
        logger.addHandler(ch)
    #use:
    # logger = log()
    # logger.warning("泰拳警告")
    # logger.info("提示")
    # logger.error("错误")
    # logger.debug("查错")
    return logger #直接返回logger
logger = log()
    # logger.warning("泰拳警告")
    # logger.info("提示")
    # logger.error("错误")
    # logger.debug("查错")


In [82]:


    
def Model(arg):
    net = Dual_Encoder(arg['n_src_vocab'],arg['src_pad_idx'],arg['class_num'],
            arg['d_word_vec'], arg['d_model'], arg['d_inner'],
            arg['n_layers'], arg['n_head'], arg['d_k'], arg['d_v'], arg['dropout'], arg['n_position'],
            )
    return net

        
def accuracy(output, target, topk=(1, )):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)
        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res



        
def train(dataLoader, net, criterion, optimizer, epoch):
    #训练
    net.train()
    
    acc_list=[]
    loss_list=[]
    for i, items in enumerate(dataLoader):
        # 将数据转移到 controller 所在 GPU
        txta_ids,txta_index,txtb_ids,txtb_index,cls=items

        optimizer.zero_grad()
        output = net(txta_ids,txta_index,txtb_ids,txtb_index)
        loss = criterion(output, cls)
        # measure accuracy and record loss
        acc, = accuracy(output, cls, topk=(1,))[0]
        loss.backward()
        optimizer.step()
        #
        acc_list.append(acc.item())
        loss_list.append(loss.item())
        
        if i==4:
            break
    return sum(acc_list)/len(acc_list),sum(loss_list)/len(loss_list)


    
def valid(val_loader, model, criterion):
    # switch to evaluate mode
    model.eval()
    
    acc_list=[]
    loss_list=[]
    with torch.no_grad():
        for i, items in enumerate(val_loader):
            # compute output
            txta_ids,txta_index,txtb_ids,txtb_index,cls=items
            output = model(txta_ids,txta_index,txtb_ids,txtb_index)
            loss = criterion(output, cls)
            # measure accuracy and record loss
            acc= accuracy(output, cls, topk=(1, ))[0]
            acc_list.append(acc.item())
            loss_list.append(loss.item())
            if i==4:
                break
    return sum(acc_list)/len(acc_list),sum(loss_list)/len(loss_list)



# 新增2：从外面得到local_rank参数，在调用DDP的时候，其会自动给出这个参数，后面还会介绍。所以不用考虑太多，照着抄就是了。
#       argparse是python的一个系统库，用来处理命令行调用，如果不熟悉，可以稍微百度一下，很简单！
parser = argparse.ArgumentParser()
parser.add_argument('--local_rank', default=-1, type=int,
                    help='node rank for distributed training')



def main(args):
    ipt_args = parser.parse_args()
    main_worker(ipt_args.local_rank, 4, args)
    
def main_worker(local_rank,ngpus_per_node,args):
    #args:config
    # pad_data_list,pad_mask_list,all_data_index,label_list=snli_data_reader(dev_path,0)
    
    train_dataset=SNLI_Dataset(args['test_path'],args['label_dict'],args['src_pad_idx'])
    valid_dataset=SNLI_Dataset(args['dev_path'],args['label_dict'],args['src_pad_idx'])
#     test_dataset=SNLI_Dataset(args['test_path'],label_dict,args['src_pad_idx'])

#     args['n_position']=train_dataset.data_max_len()
    
    batch_size = int(args['batch_size'] / ngpus_per_node)
    #设置dalaloder
    TrainLoader=DataLoader(train_dataset, batch_size=batch_size,shuffle=True,pin_memory=True)
    ValidLoader= DataLoader(valid_dataset,batch_size=batch_size,shuffle=False,pin_memory=True)
#     TestLoader= DataLoader(test_dataset,batch_size=args['batch_size'],shuffle=False,pin_memory=True)
    
    #配置网络和设置
    net=Model(args)
    if torch.cuda.is_available() and args['use_gpu']:
        #使用GPU多卡训练
        #1.初始化
        torch.distributed.init_process_group(backend="nccl")
        # 2）GPU可用则配置每个进程的gpu
        local_rank = torch.distributed.get_rank()
        torch.cuda.set_device(local_rank)
        device = torch.device("cuda",local_rank)

        # 3）使用DistributedSampler
        TrainLoader=DataLoader(train_dataset, batch_size=batch_size, shuffle=True,sampler=DistributedSampler(train_dataset))
        
        # 4) 封装之前要把模型移到对应的gpu
        net.to(device)
        if torch.cuda.device_count() > 1:
            print('Total:use', torch.cuda.device_count(), "GPUs!")
            # 5) 封装
            net = torch.nn.parallel.DistributedDataParallel(net,
                                                            device_ids=[local_rank],
                                                            output_device=local_rank)
    else:
        #使用cpu
        device = torch.device("cpu")

        net.to(device)
    
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    criterion = nn.NLLLoss()
    #开始训练epoch轮
    best_acc = 0
    iterator = tqdm(range(args['epoch']))
    logger.info('****Start Train')
    train_info={
        'train_accs':[],
        'train_loss':[],
        'valid_accs':[],
        'valid_loss':[],
    }

    for epoch in iterator:
        if args['use_gpu']:
        # DDP：设置sampler的epoch，DistributedSampler需要这个来维持各个进程之间的相同随机数种子
            TrainLoader.sampler.set_epoch(epoch)
        train_acc,train_loss = train(TrainLoader,net,criterion,optimizer,epoch)
        logger.debug('Train,epoch:%s,acc:%s,loss:%s',epoch,train_acc,train_loss)
        train_info['train_accs'].append(train_acc)
        train_info['train_loss'].append(train_loss)
        
        valid_acc,valid_loss = valid(ValidLoader,net,criterion)
        logger.debug('Valid,epoch:%s,acc:%s,loss:%s',epoch,valid_acc,valid_loss)
        train_info['valid_accs'].append(valid_acc)
        train_info['valid_loss'].append(valid_loss)
        
        # DDP:
        # 1. save模型的时候，和DP模式一样，有一个需要注意的点：保存的是model.module而不是model。
        #    因为model其实是DDP model，参数是被`model=DDP(model)`包起来的。
        # 2. 只需要在进程0上保存一次就行了，避免多次保存重复的东西。
        #请设置为只输出rank=0的进程,logging请设置为只输出rank=0的进程
        
        if args['use_gpu']:
            if torch.distributed.get_rank() == 0:
                logger.info('Train,epoch:%s,acc:%s,loss:%s',epoch,train_acc,train_loss)
                logger.info('Valid,epoch:%s,acc:%s,loss:%s',epoch,valid_acc,valid_loss)
                

                if best_acc<valid_acc:
                    
                    state={
                    'epoch': epoch + 1,
                    'state_dict': net.module.state_dict(),
                    'best_acc1': best_acc,
                    }
                    filepath="ckpt_model_%s.pth"%(valid_acc)
                    torch.save(state, filepath)
                    best_acc=valid_acc
                    
                    logger.info('ACC：%s;Best model saved!',best_acc)
                    logger.info(filepath)
                    
        else:
                if best_acc<valid_acc:
                    state={
                    'epoch': epoch + 1,
                    'state_dict': net.state_dict(),
                    'best_acc1': best_acc,
                    }
                    
                    filepath="ckpt_model_%s.pth"%(valid_acc)
                    torch.save(state, filepath)
                    best_acc=valid_acc
                    
                    logger.info('ACC：%s;Best model saved!',best_acc)
                    logger.info(filepath)
    
    logger.info(train_info)
    logger.info('****END Train')


In [63]:
# a=torch.from_numpy(np.array([[-1.5106, -1.1850, -0.7476],[-1.6361, -1.4461, -0.5625]]))
# b=torch.ones(2)
# accuracy(a,b,topk=(1, ))[0]
# torch.from_numpy(np.array([2.11])).item()

2.11

In [83]:
snli_train_config={
    'train_path':'../../data/raw_data/snli_1.0_train.jsonl',
    'test_path':'../../data/raw_data/snli_1.0_test.jsonl',
    'dev_path':'../../data/raw_data/snli_1.0_dev.jsonl',
    'label_dict':{  "entailment": 0,
                    "neutral": 1,
                    "contradiction": 2},
    #encoder_config
    'n_src_vocab':tokenizer.vocab_size,
    'src_pad_idx':0,
    'd_word_vec':512,
    'd_model':512,
    'd_inner':2048,
    'n_layers':2,
    'n_head':8,
    'd_k':64,
    'd_v':64,
    'dropout':0.1,
    'n_position':512,
    'class_num':3,
    #
    'use_gpu':False,
    'epoch':2,
    'batch_size':2,
}
main_worker(-1, 1, snli_train_config)

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]136 2020/10/07 09:42:10 info <ipython-input-82-b641ff3641b8> INFO ****Start Train 
INFO:info:****Start Train
149 2020/10/07 09:42:15 info <ipython-input-82-b641ff3641b8> DEBUG Train,epoch:0,acc:60.0,loss:3.8444475710391997 
DEBUG:info:Train,epoch:0,acc:60.0,loss:3.8444475710391997
154 2020/10/07 09:42:17 info <ipython-input-82-b641ff3641b8> DEBUG Valid,epoch:0,acc:40.0,loss:3.86313068009913 
DEBUG:info:Valid,epoch:0,acc:40.0,loss:3.86313068009913
196 2020/10/07 09:42:17 info <ipython-input-82-b641ff3641b8> INFO ACC：40.0;Best model saved! 
INFO:info:ACC：40.0;Best model saved!
197 2020/10/07 09:42:17 info <ipython-input-82-b641ff3641b8> INFO ckpt_model_40.0.pth 
INFO:info:ckpt_model_40.0.pth
 50%|██████████████████████████████████████████                                          | 1/2 [00:06<00:06,  6.92s/it]149 2020/10/07 09:42:22 info <ipython-input-82-b641ff3641b8> DE

In [77]:
a={"train_accs": [30.0, 30.0], "train_loss": [4.825904059410095, 2.101485085487366], "valid_accs": [30.0, 40.0], "valid_loss": [2.218314456939697, 2.3011699616909027]}
logger.info(a)

2 2020/10/07 09:38:20 info <ipython-input-77-f3774758c21b> INFO {'train_accs': [30.0, 30.0], 'train_loss': [4.825904059410095, 2.101485085487366], 'valid_accs': [30.0, 40.0], 'valid_loss': [2.218314456939697, 2.3011699616909027]} 
INFO:info:{'train_accs': [30.0, 30.0], 'train_loss': [4.825904059410095, 2.101485085487366], 'valid_accs': [30.0, 40.0], 'valid_loss': [2.218314456939697, 2.3011699616909027]}


In [29]:

# net.init_weight()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
criterion = nn.NLLLoss()
# criterion = nn.CrossEntropyLoss() #损失函数为交叉熵，多用于多分类问题 CrossEntropyLoss = log_sofrmax + NLLLoss

for epoch in range(10):
    for i, items in enumerate(dataLoader):
        txta_ids,txta_index,txtb_ids,txtb_index,cls=[item.to(device) for item in  items]
        
        optimizer.zero_grad()
        out = net(txta_ids,txta_index,txtb_ids,txtb_index)
        loss = criterion(out, cls)
        loss.backward()
        optimizer.step()

        if (i + 1) % 1 == 0:
            print("epoch:", epoch + 1, "step:", i + 1, "loss:", loss.item())
            data = str(epoch + 1) + ' ' + str(i + 1) + ' ' + str(loss.item()) + '\n'

torch.Size([1, 1024])
epoch: 1 step: 1 loss: 0.475547730922699
torch.Size([1, 1024])
epoch: 1 step: 2 loss: 18.809850692749023
torch.Size([1, 1024])
epoch: 1 step: 3 loss: 0.014578018337488174
torch.Size([1, 1024])
epoch: 1 step: 4 loss: 0.11480844765901566
torch.Size([1, 1024])
epoch: 1 step: 5 loss: 9.84885025024414
torch.Size([1, 1024])
epoch: 1 step: 6 loss: 10.247902870178223
torch.Size([1, 1024])
epoch: 1 step: 7 loss: 2.9223175048828125
torch.Size([1, 1024])
epoch: 1 step: 8 loss: 0.010046628303825855
torch.Size([1, 1024])
epoch: 1 step: 9 loss: 3.886147169396281e-05
torch.Size([1, 1024])
epoch: 1 step: 10 loss: 23.114463806152344
torch.Size([1, 1024])
epoch: 1 step: 11 loss: 22.879013061523438
torch.Size([1, 1024])
epoch: 1 step: 12 loss: 14.159449577331543
torch.Size([1, 1024])
epoch: 1 step: 13 loss: 9.16677454370074e-05
torch.Size([1, 1024])
epoch: 1 step: 14 loss: 5.0429368019104
torch.Size([1, 1024])
epoch: 1 step: 15 loss: 0.1432262510061264
torch.Size([1, 1024])
epoch: 1

KeyboardInterrupt: 

In [30]:


def prepare_SNLI_dataloaders(opt, device):
    batch_size = opt.batch_size
    data = pickle.load(open(opt.data_pkl, 'rb'))

    opt.max_token_seq_len = data['settings'].max_len
    opt.src_pad_idx = data['vocab']['src'].vocab.stoi[Constants.PAD_WORD]
    opt.trg_pad_idx = data['vocab']['trg'].vocab.stoi[Constants.PAD_WORD]

    opt.src_vocab_size = len(data['vocab']['src'].vocab)
    opt.trg_vocab_size = len(data['vocab']['trg'].vocab)

    #========= Preparing Model =========#
    if opt.embs_share_weight:
        assert data['vocab']['src'].vocab.stoi == data['vocab']['trg'].vocab.stoi, \
            'To sharing word embedding the src/trg word2idx table shall be the same.'

    fields = {'src': data['vocab']['src'], 'trg':data['vocab']['trg']}
    
    fields={'src':data[]}
    train = Dataset(examples=data['train'], fields=fields)
    val = Dataset(examples=data['valid'], fields=fields)
    
    
    train_iterator = BucketIterator(train, batch_size=batch_size, device=device, train=True)
    val_iterator = BucketIterator(val, batch_size=batch_size, device=device)

    return train_iterator, val_iterator
train_path='../../data/raw_data/snli_1.0_train.jsonl'
test_path='../../data/raw_data/snli_1.0_test.jsonl'
dev_path='../../data/raw_data/snli_1.0_dev.jsonl'
# folder='../data/preprocessed/SNLI/'
load_data_from_file(dev_path)
tokenizer("How old are you?", "I'm 6 years old",return_tensors="pt")



[{'index': 0,
  'sentence1': 'Two women are embracing while holding to go packages.',
  'sentence2': 'The sisters are hugging goodbye while holding to go packages after just eating lunch.',
  'gold_label': 'neutral'},
 {'index': 1,
  'sentence1': 'Two women are embracing while holding to go packages.',
  'sentence2': 'Two woman are holding packages.',
  'gold_label': 'entailment'},
 {'index': 2,
  'sentence1': 'Two women are embracing while holding to go packages.',
  'sentence2': 'The men are fighting outside a deli.',
  'gold_label': 'contradiction'},
 {'index': 3,
  'sentence1': 'Two young children in blue jerseys, one with the number 9 and one with the number 2 are standing on wooden steps in a bathroom and washing their hands in a sink.',
  'sentence2': 'Two kids in numbered jerseys wash their hands.',
  'gold_label': 'entailment'},
 {'index': 4,
  'sentence1': 'Two young children in blue jerseys, one with the number 9 and one with the number 2 are standing on wooden steps in a ba

In [190]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."

encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)
encoded_input

{'input_ids': tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [39]:
from transformers import BertTokenizer, BertModel
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
batch_sentences = ["Two young children in blue jerseys, one with the number 9 and one with the number 2 are standing on wooden steps in a bathroom and washing their hands in a sink.",
                   "And another sentence happy end",
                   "And the very very last one"]
batch_of_second_sentences = ["I'm a sentence that goes with the first sentence",
                             "And I should be encoded with the second sentence",
                             "And I go with the very last one"]
encoded_inputs = tokenizer(batch_sentences, batch_of_second_sentences)
print(encoded_inputs)

{'input_ids': [[101, 2048, 2402, 2336, 1999, 2630, 28772, 1010, 2028, 2007, 1996, 2193, 1023, 1998, 2028, 2007, 1996, 2193, 1016, 2024, 3061, 2006, 4799, 4084, 1999, 1037, 5723, 1998, 12699, 2037, 2398, 1999, 1037, 7752, 1012, 102, 1045, 1005, 1049, 1037, 6251, 2008, 3632, 2007, 1996, 2034, 6251, 102], [101, 1998, 2178, 6251, 3407, 2203, 102, 1998, 1045, 2323, 2022, 12359, 2007, 1996, 2117, 6251, 102], [101, 1998, 1996, 2200, 2200, 2197, 2028, 102, 1998, 1045, 2175, 2007, 1996, 2200, 2197, 2028, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1

In [15]:
import torch
import numpy as np
# a=torch.tensor([[[5,5,5,5], [6,6,6,6], [7,7,7,7]], [[1,1,1,1],[2,2,2,2],[3,3,3,3]]])
a=torch.arange(1,40).view(3,13)
a[a==1]
# print(a)
# print(a.size())
# print("#############################################3")
# mask = torch.ByteTensor([[[1],[1],[0]],[[0],[1],[1]]])
# print(mask)
# print(mask.size())
# b = a.masked_fill(mask, value=torch.tensor(-1e9))
# print(b)
# print(b.size())

tensor([1])

In [40]:
# device = torch.device('cuda' if torch.cuda.is_available()  else 'cpu')
# # N是批量大小，D_in 是输入维度，H 是隐藏维度， D_out 是输出维度
# # data=torch.randn()


# # input_data = data.to
# n_src_vocab=1000
# src_pad_idx=0

# d_word_vec=512
# d_model=512
# d_inner=2048
# n_layers=6
# n_head=8
# d_k=64
# d_v=64
# dropout=0.1
# n_position=200

# sup_enc = Encoder(n_src_vocab = n_src_vocab, n_position = n_position,
#         d_word_vec = d_word_vec, d_model=d_model, d_inner=d_inner,
#         n_layers=n_layers, n_head=n_head, d_k=d_k, d_v=d_v,
#         pad_idx=src_pad_idx, dropout=dropout)
# # sub_enc

# # input=torch.autograd.Variable(input)
# # sup_enc(input)

# #batch,seq_len
# # 3，8，10，10 - 3,1,10

# data=torch.arange(0,3*8*10*10).view(3,8,10,10).long()    #三个句子，每个句子有两个词

# mask=torch.ones(3,6).view(3,6,1)
# zero_append=torch.zeros(3,4).view(3,4,1)
# mask=torch.cat([mask,zero_append],1)
# mask.shape
# data.masked_fill(mask, value=torch.tensor(-1e9))
# # sup_enc(data,mask)

# # data.masked_fill(mask == 0, -1e9)